In [1]:
print("ok")

ok


In [2]:
%pwd

'c:\\Users\\Vinay Maurya\\Desktop\\Medical Chatbot\\Medical-Chatbot-Generative-AI\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Vinay Maurya\\Desktop\\Medical Chatbot\\Medical-Chatbot-Generative-AI'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
#Extract Data from the PDF Files

def load_pdf_files(data):
    loader=DirectoryLoader(data,
                           glob="*.pdf",
                           loader_cls=PyPDFLoader)
    
    documents=loader.load()

    return documents

In [7]:
extracted_data=load_pdf_files(data='Data/')

In [10]:
# extracted_data

In [8]:
#Split the Data into Text Chunks

def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [9]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks",len(text_chunks))

Length of Text Chunks 40000


In [15]:
# text_chunks

In [10]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

In [11]:
def download_hugging_face_embeddings():
    embeddings=HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [12]:
embeddings=download_hugging_face_embeddings()

C:\Users\Vinay Maurya\AppData\Local\Temp\ipykernel_13372\2220115462.py:2: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\Users\Vinay Maurya\Desktop\Medical Chatbot\Medical-Chatbot-Generative-AI\medibot\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
query_results=embeddings.embed_query("Hello World")
print("Length",len(query_results))

Length 384


In [25]:
# query_results

In [14]:
from dotenv import load_dotenv
load_dotenv()

True

In [15]:
import os

PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [34]:
# PINECONE_API_KEY

In [22]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

pc=Pinecone(api_key=PINECONE_API_KEY)

index_name="medical-bot"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

{
    "name": "medical-bot",
    "metric": "cosine",
    "host": "medical-bot-i2or3u7.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [23]:
from pinecone import Pinecone

pc = Pinecone(api_key="PINECONE_API_KEY")


In [24]:
import os

os.environ["PINECONE_API_KEY"]=PINECONE_API_KEY
os.environ["OPENAI_API_KEY"]=OPENAI_API_KEY

In [25]:
from langchain_pinecone import PineconeVectorStore

docsearch=PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings
)

In [26]:
#Load Exiting Index

from langchain_pinecone import PineconeVectorStore

#Embed each chunk and upsert the emebbings into your Pinecone index.

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [27]:
retriever=docsearch.as_retriever(search_type="similarity",search_kwargs={"k":3})

In [60]:
retriever_docs=retriever.invoke("What is fever?")

In [62]:
retriever_docs

[Document(id='0b2efd47-62d5-42be-b45e-61c11e3f3d7e', metadata={'creationdate': '2006-10-16T20:19:33+02:00', 'creator': 'Adobe Acrobat 6.0', 'moddate': '2006-10-16T22:03:45+02:00', 'page': 1493.0, 'page_label': '1464', 'producer': 'PDFlib+PDI 6.0.3 (SunOS)', 'source': 'Data\\Test_Data.pdf', 'total_pages': 4505.0}, page_content='Diagnosis\nFew symptoms in medicine present such a diagnos-\ntic challenge as fever. Nonetheless, if a careful, logical,\nand thorough evaluation is performed, a diagnosis will\nbe found in most cases. The patient’s past medical\nhistory as well as travel, social, and family history\nshould be carefully searched for important clues.\nUsually the first step is to search for an infectious\ncause. Skin and other screening tests for diseases such\nas tuberculosis, and examination of blood, urine, and'),
 Document(id='4788b7a6-02a3-43da-9f9e-185e81c4b435', metadata={'creationdate': '2006-10-16T20:19:33+02:00', 'creator': 'Adobe Acrobat 6.0', 'moddate': '2006-10-16T22:

In [47]:
from langchain_openai import OpenAI
llm=OpenAI(temperature=0.0,max_tokens=500)

In [48]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt=(
    "You are an assistant for question-answering tasks."
    "Use the following pieces of retrieved context to answer"
    "The question. If you don't know the answer, say that you"
    "Don't Know. Use three sentences maximum and keep the "
    "answer concise"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}")
    ]
)

In [55]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain= create_retrieval_chain(retriever,question_answer_chain)

In [65]:
response=rag_chain.invoke({"input": "What is fever?"})
print(response["answer"])



Fever is a natural response of the body to fight off foreign substances. It is controlled by the thermoregulatory center in the brain and varies throughout the day. A thorough evaluation is necessary to determine the cause of fever, which is usually an infectious disease.


In [71]:
response=rag_chain.invoke({"input": "What is Javascript "})
print(response["answer"])



Don't Know.
